In [1]:
from cifkit import Cif
cif = Cif("Yb14MnSb11.cif") 
cif.compute_connections()

In [12]:
from scipy.spatial import ConvexHull
from cifkit.coordination.geometry import get_polyhedron_coordinates_labels, compute_center_of_mass_and_distance
from cifkit.figures import polyhedron
from cifkit.coordination.geometry import compute_polyhedron_metrics

# Start from Mn site
site_label = "Mn"
# Get the first 4 nearest neighbors from Mn
polyhedron_points, vertex_labels = cif.get_polyhedron_labels_by_CN_best_methods(site_label)

# Print the last element of the vertex_labels list
processed_vertex_labels = vertex_labels[:4] + [vertex_labels[-1]]
processed_polyhedron_points = polyhedron_points[:4] + [polyhedron_points[-1]]
hull = ConvexHull(processed_polyhedron_points)

def compute_polyhedron_metrics(polyhedron_points, hull):
    """Compute various metrics related to a given polyhedron."""
    central_atom_coord = np.array(polyhedron_points[-1])

    # Convert to NumPy array excluding the last element
    polyhedron_points = np.array(polyhedron_points[:-1])

    _, distance_to_center = compute_center_of_mass_and_distance(
        polyhedron_points, hull, central_atom_coord
    )

    edges = set()
    edge_distances = []
    
    for simplex in hull.simplices:
        for i in range(-1, len(simplex) - 1):
            # Create a sorted tuple to ensure uniqueness (avoid duplicates in reverse order)
            edge = tuple(sorted([simplex[i], simplex[i + 1]]))
            if edge not in edges:
                edges.add(edge)
                # Calculate the distance
                distance = np.linalg.norm(polyhedron_points[edge[0]] - polyhedron_points[edge[1]])
                edge_distances.append((edge, distance))
    # Basic polyhedron info
    number_of_edges = len(edges)
    # Sort the edges by distance and get the shortest and longest
    edge_distances = sorted(edge_distances, key=lambda x: x[1])
    shortest_edge = edge_distances[0][1]
    longest_edge = edge_distances[-1][1]
    
    # print("Mn central atom, 4 nearest neighbors\n")
    # print("Connection information:")
    # print("Neighbor atom, distance, central_atom coord, other_atom_coord")
    # for connection in cif.connections[site_label][:4]:
    #     print(connection)
    
    # print("\nEdge information:")
    # print("Node 1, Node 2, Distance")
    # for edge, distance in edge_distances:
    #     print(edge, distance)

    print("\nFeature information:")
    print("1. Vol of polyhedron:", np.round(hull.volume, 3))
    print("2. Dist from central atom to center of mass:", distance_to_center)
    print("3. Avg dist of edges:",sum([distance for _, distance in edge_distances])/number_of_edges)
    print("4. Distortion of edges (shortest edge divided by longest):", shortest_edge/longest_edge)
    
compute_polyhedron_metrics(processed_polyhedron_points, hull)



Feature information:
1. Vol of polyhedron: 10.299
2. Dist from central atom to center of mass: 0.0008660254037834459
3. Avg dist of edges: 4.4575654833676
4. Distortion of edges (shortest edge divided by longest): 0.93163394769286


In [19]:
from scipy.spatial import ConvexHull
from cifkit.coordination.geometry import get_polyhedron_coordinates_labels, compute_center_of_mass_and_distance
from cifkit.figures import polyhedron
from cifkit.coordination.geometry import compute_polyhedron_metrics
from skspatial.objects import Plane, Points
from skspatial.plotting import plot_3d

# Start from Sb4 site
site_label = "Sb4"
CN = 10
# Get the first 10 nearest neighbors from Mn
polyhedron_points, vertex_labels = cif.get_polyhedron_labels_by_CN_best_methods(site_label)

# Print the last element of the vertex_labels list
processed_vertex_labels = vertex_labels[:CN] + [vertex_labels[-1]]
processed_polyhedron_points = polyhedron_points[:CN] + [polyhedron_points[-1]]
hull = ConvexHull(processed_polyhedron_points)

def compute_polyhedron_metrics(polyhedron_points, hull):
    """Compute various metrics related to a given polyhedron."""
    central_atom_coord = np.array(polyhedron_points[-1])

    # Convert to NumPy array excluding the last element
    polyhedron_points = np.array(polyhedron_points[:-1])

    _, distance_to_center = compute_center_of_mass_and_distance(
        polyhedron_points, hull, central_atom_coord
    )

    edges = set()
    edge_distances = []
    
    for simplex in hull.simplices:
        for i in range(-1, len(simplex) - 1):
            # Create a sorted tuple to ensure uniqueness (avoid duplicates in reverse order)
            edge = tuple(sorted([simplex[i], simplex[i + 1]]))
            if edge not in edges:
                edges.add(edge)
                # Calculate the distance
                distance = np.linalg.norm(polyhedron_points[edge[0]] - polyhedron_points[edge[1]])
                edge_distances.append((edge, distance))
    # Basic polyhedron info
    number_of_edges = len(edges)
    # Sort the edges by distance and get the shortest and longest
    edge_distances = sorted(edge_distances, key=lambda x: x[1])
    shortest_edge = edge_distances[0][1]
    longest_edge = edge_distances[-1][1]
    
    # print(f"Sb4 central atom, {CN} nearest neighbors (Sb4 assumed to be central here\n")
    
    # print("Connection information:")
    # print("Neighbor atom, distance, central_atom coord, other_atom_coord")
    # for connection in cif.connections[site_label][:CN]:
    #     print(connection)
    
    # print("\nEdge information:")
    # print("Node 1, Node 2, Distance")
    # for edge, distance in edge_distances:
    #     print(edge, distance)
        
    # (7) Avg distance from the central Sb to Yb1
    sb_yb1_total_distance = float()
    sb_yb1_neighbor_count = int()
    for connection in cif.connections[site_label][:CN]:
        if connection[0] == "Yb1":
            sb_yb1_total_distance += connection[1]
            sb_yb1_neighbor_count += 1
    avg_dist_sb_yb1 = sb_yb1_total_distance / sb_yb1_neighbor_count
    
    # (8) Avg distance from central Sb to Yb2
    sb_yb2_total_distance = float()
    sb_yb2_neighbor_count = int()
    for connection in cif.connections[site_label][:CN]:
        if connection[0] == "Yb2":
            sb_yb2_total_distance += connection[1]
            sb_yb2_neighbor_count += 1
    avg_dist_sb_yb2 = sb_yb2_total_distance / sb_yb2_neighbor_count
    
    # (9) Avg distance from the central atom to Yb
    sb_yb_total_distance = float()
    sb_yb_neighbor_count = int()
    for connection in cif.connections[site_label][:CN]:
        if connection[0].startswith("Yb"):
            sb_yb_total_distance += connection[1]
            sb_yb_neighbor_count += 1
    avg_dist_sb_yb = sb_yb_total_distance / sb_yb_neighbor_count

    # (10) shortest Sb-Yb to longest Sb-Yb
    sb_yb_shortest_distance = float("inf")
    sb_yb_longest_distance = float("-inf")
    for connection in cif.connections[site_label][:CN]:
        if connection[0].startswith("Yb"):
            if connection[1] < sb_yb_shortest_distance:
                sb_yb_shortest_distance = connection[1]
            if connection[1] > sb_yb_longest_distance:
                sb_yb_longest_distance = connection[1]
    shortest_by_longest_sb_yb = sb_yb_shortest_distance / sb_yb_longest_distance
    
    # (11) distortion from ideal square Yb2Yb1Yb2Yb1
    # Find the four nearest neighbors of Sb1
    # There are two labels of Sb1, but let's just use the very first one=
    sb1_coord = None
    sb4_central_atom_10_connections = cif.connections[site_label][:CN]
    for connection in sb4_central_atom_10_connections:
        if connection[0] == "Sb1":
            sb1_coord = connection[3]
            break
    # Compute the distance from Sb1
    dist_label_from_sb1 = {}
    neighbot_counter_from_sb1 = 1
    for connection in sb4_central_atom_10_connections:
        if connection[0] != "Sb1":
            distance = np.linalg.norm(np.array(connection[3]) - np.array(sb1_coord))
            neighbor_coord = connection[3]
            dist_label_from_sb1[neighbot_counter_from_sb1] = {
                "distance": distance,
                "site_label": connection[0],
                "cart_coord": neighbor_coord
            }
        
            neighbot_counter_from_sb1 += 1
    
    # Sort the label counter based on the shortest distance
    sorted_dist_label_from_sb1 = sorted(dist_label_from_sb1.items(), key=lambda x: x[1]["distance"])
    # Compute the four nearest neighbors of Sb1
    sb1_four_nearest_neighbors_coords = Points([dist_label_from_sb1[1]["cart_coord"], dist_label_from_sb1[2]["cart_coord"], dist_label_from_sb1[3]["cart_coord"], dist_label_from_sb1[4]["cart_coord"]])
    sb1_four_nearest_neighbors_plane = Plane.best_fit(sb1_four_nearest_neighbors_coords)
    sb1_four_nearest_neighbors_norm_vector = sb1_four_nearest_neighbors_plane.normal

    # Compute the three nearest neighbors of Sb1
    sb1_three_nearest_neighbors_coords = Points([dist_label_from_sb1[1]["cart_coord"], dist_label_from_sb1[2]["cart_coord"], dist_label_from_sb1[3]["cart_coord"]])
    sb1_three_nearest_neighbors_plane = Plane.best_fit(sb1_three_nearest_neighbors_coords)
    sb1_three_nearest_neighbors_norm_vector = sb1_three_nearest_neighbors_plane.normal
    assert np.array_equal(sb1_three_nearest_neighbors_norm_vector, sb1_four_nearest_neighbors_norm_vector) == False
    
    # Use the dot product to compute the angle
    angle = np.arccos(np.dot(sb1_four_nearest_neighbors_norm_vector, sb1_three_nearest_neighbors_norm_vector))
    
    # 5 through 11 features
    print("5. Vol of polyhedron:", np.round(hull.volume, 3))
    print("6. Distance from central atom to center of mass:", np.around(distance_to_center, 3))
    print("7. Avg distance from the central Sb to Yb1", avg_dist_sb_yb1)
    print("8. Avg distance from the central Sb to Yb2", avg_dist_sb_yb2)
    print("9. Avg distance from the cental atom to Yb", np.round(avg_dist_sb_yb, 3))
    print("10. Shortest Sb-Yb to the longest Sb-Yb in", np.round(shortest_by_longest_sb_yb, 3))
    print("11. Distortion from the ideal square Yb2Yb1Yb2Yb1", np.round(angle, 3))
    
compute_polyhedron_metrics(processed_polyhedron_points, hull)


5. Vol of polyhedron: 77.572
6. Distance from central atom to center of mass: 0.001
7. Avg distance from the central Sb to Yb1 3.176
8. Avg distance from the central Sb to Yb2 3.411
9. Avg distance from the cental atom to Yb 3.294
10. Shortest Sb-Yb to the longest Sb-Yb in 0.931
11. Distortion from the ideal square Yb2Yb1Yb2Yb1 0.658
